In [1]:
# # Import relevant functionality
# from langchain.chat_models import init_chat_model
# from langchain_tavily import TavilySearch
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent

# # Create the agent
# memory = MemorySaver()
# model = init_chat_model("anthropic:claude-3-5-sonnet-latest")
# search = TavilySearch(max_results=2)
# tools = [search]
# agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [2]:
# import getpass
# import os

# os.environ["LANGSMITH_TRACING"] = "true"
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass()

In [3]:
import getpass
import os
from dotenv import find_dotenv, load_dotenv
# Load environment variables
load_dotenv(find_dotenv())

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain.chat_models import init_chat_model

model = init_chat_model("gpt-4.1", model_provider="openai")

## make model agnostic

In [4]:
query = "Hi!"
response = model.invoke([{"role": "user", "content": query}])
response.text()

'Hello! How can I help you today? 😊'

In [5]:
from langchain_core.tools import tool

@tool
def multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

@tool
def divide(a: int, b: int) -> int:
    """Divide two numbers."""
    return a / b

tools = [multiply, divide]

## add tools

In [6]:
model_with_tools = model.bind_tools(tools)

In [7]:
query = "what is the product of 2 times 3"
response = model_with_tools.invoke([{"role": "user", "content": query}])

print(f"Message content: {response.text()}\n")
print(f"Tool calls: {response.tool_calls}")

Message content: 

Tool calls: [{'name': 'multiply', 'args': {'a': 2, 'b': 3}, 'id': 'call_y84BGR6Q2BBsR2q9h5SSFacY', 'type': 'tool_call'}]


In [8]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [24]:
query = "what is the product of 2 times 3"
input_message = {"role": "user", "content": query}
response = agent_executor.invoke({"messages": [input_message]})

## Show results all at once
# for message in response["messages"]:
#     message.pretty_print()

# Show immediate message
for step in agent_executor.stream({"messages": [input_message]}, stream_mode="values"):
    step["messages"][-1].pretty_print()


================================ Human Message =================================

what is the product of 2 times 3
================================== Ai Message ==================================
Tool Calls:
  multiply (call_CK1nYXcBsC2NNe60u8YMVtM6)
 Call ID: call_CK1nYXcBsC2NNe60u8YMVtM6
  Args:
    a: 2
    b: 3
================================= Tool Message =================================
Name: multiply

6
================================== Ai Message ==================================

The product of 2 times 3 is 6.


In [9]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [10]:
query = "Can you get the product of 115 by 5? Then, divide the answer by 10. Who owns the dog?"

for step in agent_executor.stream(
    {"messages": [("user", query)]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

Can you get the product of 115 by 5? Then, divide the answer by 10. Who owns the dog?
================================== Ai Message ==================================
Tool Calls:
  multiply (call_zKCAxeewUwS71WVQRKjsTH0k)
 Call ID: call_zKCAxeewUwS71WVQRKjsTH0k
  Args:
    a: 115
    b: 5
================================= Tool Message =================================
Name: multiply

575
================================== Ai Message ==================================
Tool Calls:
  divide (call_Zm9x9PqeFhzZrrQr7TuBnWaU)
 Call ID: call_Zm9x9PqeFhzZrrQr7TuBnWaU
  Args:
    a: 575
    b: 10
================================= Tool Message =================================
Name: divide

57.5
================================== Ai Message ==================================

The product of 115 by 5 is 575. When you divide 575 by 10, you get 57.5. As for "who owns the dog," that information hasn't been specified—ple

In [60]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


# system_prompt = """
# You are operations person with extensive years of experience. Provide analysis in detail. Do not make up any answers. Say I don't know if you do not have the answers.

# Always think of production thoughput is king on decisions!
# """

system_prompt = """
You are an electrical engineer with extensive years of experience. 
Ensure safety and reliability is maintained on decisions!

Avoid using numbered bullets or list. Answer in concise paragraphs and thorough response. 
Provide analysis in detail. Do not make up any answers. Say I don't know if you do not have the answers.
"""

agent_executor = create_react_agent(model, tools, checkpointer=memory, prompt=system_prompt)

config = {"configurable": {"thread_id": "2"}}

In [61]:
query = "I have a AG mill. Do you recommend to push it hard to gain production increase?"

for step in agent_executor.stream(
    {"messages": [("user", query)]}, config, stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

I have a AG mill. Do you recommend to push it hard to gain production increase?
================================== Ai Message ==================================

Increasing throughput in an AG (Autogenous Grinding) mill to boost production is tempting, but the decision requires careful analysis. AG mills have distinct operational limits governed by mill motor power, mechanical integrity, liner/lifter wear, ore characteristics, and downstream process capacity.

Pushing an AG mill “hard” can indeed lead to higher short-term production, but if you approach or exceed motor power availability, you risk tripping the drive or overheating components, which can cause long downtimes. Overloading can also result in excessive wear on linings and lifters, unplanned maintenance, and possible shell or trunnion cracks. Throughput increases that do not match downstream capability can create bottlenecks or overload other c

In [ ]:
def research_agent(task: str, model: str = "gpt-5"):
    """
    Ejecuta una tarea de investigación usando herramientas con aisuite (sin bucle manual).
    """
    print("==================================")
    print("🔍 Research Agent")
    print("==================================")
    max_tool_call = 3
    tool_calls = 0
    prompt = f"""
You are a research assistant with access to the following tools:
- arxiv_tool: for finding academic papers
- tavily_tool: for general web search
- wikipedia_tool: for encyclopedic knowledge

Task:
{task}

Today is {datetime.now().strftime('%Y-%m-%d')}

Limit tool calling into {max_tool_call} maximum
"""
    print(prompt)
    def run_tool(name, args):
        if name == "arxiv_search_tool":
            return research_tools.arxiv_search_tool(**args)
        if name == "tavily_search_tool":
            return research_tools.tavily_search_tool(**args)
        if name == "wikipedia_search_tool":
            return research_tools.wikipedia_search_tool(**args)
        return {"error": f"Unknown tool: {name}"}

    messages = [{"role": "user", "content":prompt.strip()}]
    tools = [research_tools.arxiv_tool_def, research_tools.tavily_tool_def, research_tools.wikipedia_tool_def]
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            tools=tools,
            tool_choice="auto"
        )

        msg = response.choices[0].message

        messages.append(msg)
        # kept = msg.tool_calls[:max_tool_call]
        # sanitized_assistant = {
        #     "role": "assistant",
        #     "content": msg.content,
        #     "tool_calls": kept,
        # }
        # messages.append(sanitized_assistant)
        for call in msg.tool_calls:
            tool_calls += 1
            print(call.function.name, call.function.arguments)
            if tool_calls <= max_tool_call:
                result = run_tool(call.function.name, json.loads(call.function.arguments))
                messages.append({
                    "role": "tool",
                    "tool_call_id": call.id,
                    "name": call.function.name,
                    "content": json.dumps(result)
                })
        # if tool_calls > 3:
        #     messages.append({
        #         "role": "assistant",
        #         "content": "I have reached the maximum tool usage as instructed. ",
        #     })
            
        final_response = client.chat.completions.create(
            model=model,
            messages=messages,
            reasoning_effort = "minimal"
        )
        
        content = final_response.choices[0].message.content
        print("✅ Output:\n", content)
        used_tokens = response.usage.total_tokens
        print("Used Tokens:\n", used_tokens)
        return content, used_tokens

    except Exception as e:
        print("❌ Error:", e)
        return f"[Model Error: {str(e)}]"
